### The basis:
https://github.com/neptune-ml/open-solution-home-credit/tree/solution-5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from scipy.stats import skew, kurtosis, iqr

import gc
import os
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)

In [2]:
PATH_TO_DATA = ('D:/Py/DataFrames/Home_Credit_Default_Risk(KAGGLE)/')

In [3]:
def only_sk_id_aggregated_features(df, new_df, recipies, col_names=True):
    group_object = df.groupby('SK_ID_CURR')
    if not col_names:
        for select, agg in tqdm(recipies):
            groupby_aggregate_name = 'by_ID_{}_{}'.format(agg, select)
            new_df = new_df.merge(group_object[select].agg(agg).reset_index()
                                                      .rename(index=str, columns={select: groupby_aggregate_name}),
                                                      on='SK_ID_CURR', how='left')
    else:
        for select, agg, col_name in tqdm(recipies):
            new_df = new_df.merge(group_object[select].agg(agg).reset_index()
                                                      .rename(index=str, columns={select: col_name}),
                                                      on='SK_ID_CURR', how='left')
    return new_df

# add Features

### Application train/test

In [22]:
app_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'application_train.csv'))
app_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'application_test.csv'))

In [23]:
# Preprocessing
for df in [app_train, app_test]:
    df['CODE_GENDER'].replace('XNA', np.nan, inplace=True)
    df['NAME_FAMILY_STATUS'].replace('Unknown', 'Single / not married', inplace=True) # just my idea
    df['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace=True)
    for col in ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']:
        df[col].replace(365243, np.nan, inplace=True)

In [24]:
# Creating simple features
for df in [app_train, app_test]:
    df['annuity_income_perc'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['car_to_birth_ratio'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['car_to_employ_ratio'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['children_ratio'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']
    df['credit_to_goods_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['credit_to_income_ratio'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['employ_bitrh_gap'] = (df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']) / -365
    #df['days_employed_perc'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['income_credit_perc'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['income_per_child'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN']) # to avoid +-np.inf
    df['income_per_person'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['payment_rate'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    df['phone_to_birth_ratio'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    df['phone_to_employ_ratio'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    
    df['cnt_non_child'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
    df['child_to_non_child_ratio'] = df['CNT_CHILDREN'] / df['cnt_non_child']
    df['income_per_non_child'] = df['AMT_INCOME_TOTAL'] / df['cnt_non_child']
    df['credit_per_person'] = df['AMT_CREDIT'] / df['CNT_FAM_MEMBERS']
    df['credit_per_child'] = df['AMT_CREDIT'] / (1 + df['CNT_CHILDREN'])
    df['credit_per_non_child'] = df['AMT_CREDIT'] / df['cnt_non_child']
    
    df['long_employment'] = (df['DAYS_EMPLOYED'] < -2000).astype(int)
    df['retirement_age'] = (df['DAYS_BIRTH'] < -14000).astype(int)

In [25]:
# Creating Features based on EXT_SOURCE_...
for df in [app_train, app_test]:
    df['external_sources'] = df['EXT_SOURCE_1'] + df['EXT_SOURCE_2'] + df['EXT_SOURCE_3']
    for function_name in ['min', 'max', 'sum', 'mean']:
        df['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name)) \
                                                            (df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

In [26]:
# Creating datasets for new features
app_train_new_features = app_train[app_train.columns.tolist()[-26:] + ['SK_ID_CURR', 'TARGET']]
app_test_new_features = app_test[app_test.columns.tolist()[-26:] + ['SK_ID_CURR']]

In [27]:
app_train_new_features.head(3)

,annuity_income_perc,car_to_birth_ratio,car_to_employ_ratio,children_ratio,credit_to_goods_ratio,credit_to_income_ratio,employ_bitrh_gap,income_credit_perc,income_per_child,income_per_person,payment_rate,phone_to_birth_ratio,phone_to_employ_ratio,cnt_non_child,child_to_non_child_ratio,income_per_non_child,credit_per_person,credit_per_child,credit_per_non_child,long_employment,retirement_age,external_sources,external_sources_min,external_sources_max,external_sources_sum,external_sources_mean,SK_ID_CURR,TARGET
0,0.121978,NaN,NaN,0.0,1.158397,2.007889,-24.175342,0.498036,202500.0,202500.0,0.060749,0.119860,1.780220,1.0,0.0,202500.0,406597.50,406597.5,406597.50,0,0,0.485361,0.083037,0.262949,0.485361,0.161787,100002,1
1,0.132217,NaN,NaN,0.0,1.145199,4.790750,-42.676712,0.208736,270000.0,135000.0,0.027598,0.049389,0.696970,2.0,0.0,135000.0,646751.25,1293502.5,646751.25,0,1,NaN,0.311267,0.622246,0.933513,0.466757,100003,0
2,0.100000,-0.001365,-0.115556,0.0,1.000000,2.000000,-51.564384,0.500000,67500.0,67500.0,0.050000,0.042791,3.622222,1.0,0.0,67500.0,135000.00,135000.0,135000.00,0,1,NaN,0.555912,0.729567,1.285479,0.642739,100004,0


In [28]:
# Let's make aggregations!
APP_AGGREGATION_RECIPIES = [
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                              ('AMT_CREDIT', 'max'),
                                              ('AMT_INCOME_TOTAL', 'mean'), # my
                                              #('EXT_SOURCE_1', 'mean'),
                                              #('EXT_SOURCE_2', 'mean'),
                                              ('external_sources_mean', 'mean'), # my
                                              ('OWN_CAR_AGE', 'max'),
                                              ('OWN_CAR_AGE', 'sum')]),
    
    (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                 ('CNT_CHILDREN', 'mean'),
                                                 ('DAYS_ID_PUBLISH', 'mean'),
                                                 ('external_sources_mean', 'mean')]),
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('external_sources_mean', 'mean'), # my (and down too)
                                                                                           ('REGION_POPULATION_RELATIVE', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE'], [('AMT_CREDIT', 'mean'),
                                                  ('AMT_REQ_CREDIT_BUREAU_YEAR', 'mean'),
                                                  ('APARTMENTS_AVG', 'mean'),
                                                  ('BASEMENTAREA_AVG', 'mean'),
                                                  #('EXT_SOURCE_1', 'mean'),
                                                  #('EXT_SOURCE_2', 'mean'),
                                                  #('EXT_SOURCE_3', 'mean'),
                                                  ('external_sources_mean', 'mean'),
                                                  ('NONLIVINGAREA_AVG', 'mean'),
                                                  ('OWN_CAR_AGE', 'mean'),
                                                  ('YEARS_BUILD_AVG', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('ELEVATORS_AVG', 'mean')]),
    
    (['OCCUPATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('CNT_CHILDREN', 'mean'),
                           ('CNT_FAM_MEMBERS', 'mean'),
                           ('DAYS_BIRTH', 'mean'),
                           ('DAYS_EMPLOYED', 'mean'),
                           ('DAYS_ID_PUBLISH', 'mean'),
                           ('DAYS_REGISTRATION', 'mean'),
                           #('EXT_SOURCE_1', 'mean'),
                           #('EXT_SOURCE_2', 'mean'),
                           #('EXT_SOURCE_3', 'mean'),
                           ('external_sources_mean', 'mean')])
]

In [29]:
def aggregated_features(df, recipies):
    groupby_aggregate_names = []
    for groupby_cols, specs in tqdm(recipies):
        group_object = df.groupby(groupby_cols)
        for select, agg in tqdm(specs):
            groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
            df = df.merge(group_object[select].agg(agg).reset_index()
                                              .rename(index=str, columns={select: groupby_aggregate_name})
                                              [groupby_cols + [groupby_aggregate_name]],
                                              how='left')
            groupby_aggregate_names.append(groupby_aggregate_name)
    return df, list(set(groupby_aggregate_names))

In [30]:
app_train, groupby_aggregate_names = aggregated_features(app_train, APP_AGGREGATION_RECIPIES)
app_test, groupby_aggregate_names = aggregated_features(app_test, APP_AGGREGATION_RECIPIES)

In [31]:
app_train.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,annuity_income_perc,car_to_birth_ratio,car_to_employ_ratio,children_ratio,credit_to_goods_ratio,credit_to_income_ratio,employ_bitrh_gap,income_credit_perc,income_per_child,income_per_person,payment_rate,phone_to_birth_ratio,phone_to_employ_ratio,cnt_non_child,child_to_non_child_ratio,income_per_non_child,credit_per_person,credit_per_child,credit_per_non_child,long_employment,retirement_age,external_sources,external_sources_min,external_sources_max,external_sources_sum,external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_ANNUITY,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT,CODE_GENDER_NAME_EDUCATION_TYPE_mean_AMT_INCOME_TOTAL,CODE_GENDER_NAME_EDUCATION_TYPE_mean_external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_max_OWN_CAR_AGE,CODE_GENDER_NAME_EDUCATION_TYPE_sum_OWN_CAR_AGE,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_REGION_POPULATION_RELATIVE,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_CREDIT,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_REQ_CREDIT_BUREAU_YEAR,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_APARTMENTS_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_BASEMENTAREA_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_external_sources_mean,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_NONLIVINGAREA_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_CAR_AGE,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_YEARS_BUILD_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_ELEVATORS_AVG,OCCUPATION_TYPE_mean_AMT_ANNUIT

**Hand crafted features**

In [32]:
def diff_aggregated_application_features(df):
    diff_feature_names = []
    for groupby_cols, specs in tqdm(APP_AGGREGATION_RECIPIES):
        for select, agg in specs:
            if agg in ['min', 'max', 'mean', 'median']:
                groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
                diff_name = '{}_diff'.format(groupby_aggregate_name)
                abs_diff_name = '{}_abs_diff'.format(groupby_aggregate_name)
                
                df[diff_name] = df[select] - df[groupby_aggregate_name]
                df[abs_diff_name] = np.abs(df[select] - df[groupby_aggregate_name])
                
                diff_feature_names.append(diff_name)
                diff_feature_names.append(abs_diff_name)
    return df, list(set(diff_feature_names))

In [33]:
app_train, diff_feature_names = diff_aggregated_application_features(app_train)
app_test, diff_feature_names = diff_aggregated_application_features(app_test)

In [34]:
app_train.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,annuity_income_perc,car_to_birth_ratio,car_to_employ_ratio,children_ratio,credit_to_goods_ratio,credit_to_income_ratio,employ_bitrh_gap,income_credit_perc,income_per_child,income_per_person,payment_rate,phone_to_birth_ratio,phone_to_employ_ratio,cnt_non_child,child_to_non_child_ratio,income_per_non_child,credit_per_person,credit_per_child,credit_per_non_child,long_employment,retirement_age,external_sources,external_sources_min,external_sources_max,external_sources_sum,external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_ANNUITY,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT,CODE_GENDER_NAME_EDUCATION_TYPE_mean_AMT_INCOME_TOTAL,CODE_GENDER_NAME_EDUCATION_TYPE_mean_external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_max_OWN_CAR_AGE,CODE_GENDER_NAME_EDUCATION_TYPE_sum_OWN_CAR_AGE,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_REGION_POPULATION_RELATIVE,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_CREDIT,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_REQ_CREDIT_BUREAU_YEAR,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_APARTMENTS_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_BASEMENTAREA_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_external_sources_mean,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_NONLIVINGAREA_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_CAR_AGE,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_YEARS_BUILD_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_ELEVATORS_AVG,OCCUPATION_TYPE_mean_AMT_ANNUIT

In [35]:
app_train.shape, app_test.shape

((307511, 233), (48744, 232))

In [36]:
# Merge
app_train_new_features = app_train_new_features.merge(app_train[groupby_aggregate_names +
                                                                diff_feature_names +
                                                                ['SK_ID_CURR']],
                                                      on='SK_ID_CURR', how='left')
app_test_new_features = app_test_new_features.merge(app_test[groupby_aggregate_names +
                                                             diff_feature_names +
                                                             ['SK_ID_CURR']],
                                                    on='SK_ID_CURR', how='left')

In [37]:
app_train_new_features.head(3)

,annuity_income_perc,car_to_birth_ratio,car_to_employ_ratio,children_ratio,credit_to_goods_ratio,credit_to_income_ratio,employ_bitrh_gap,income_credit_perc,income_per_child,income_per_person,payment_rate,phone_to_birth_ratio,phone_to_employ_ratio,cnt_non_child,child_to_non_child_ratio,income_per_non_child,credit_per_person,credit_per_child,credit_per_non_child,long_employment,retirement_age,external_sources,external_sources_min,external_sources_max,external_sources_sum,external_sources_mean,SK_ID_CURR,TARGET,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_BASEMENTAREA_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_CAR_AGE,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_ANNUITY,OCCUPATION_TYPE_mean_AMT_ANNUITY,OCCUPATION_TYPE_mean_external_sources_mean,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_CREDIT,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_NONLIVINGAREA_AVG,OCCUPATION_TYPE_mean_DAYS_EMPLOYED,OCCUPATION_TYPE_mean_CNT_FAM_MEMBERS,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean,CODE_GENDER_NAME_EDUCATION_TYPE_max_OWN_CAR_AGE,OCCUPATION_TYPE_mean_DAYS_ID_PUBLISH,CODE_GENDER_NAME_EDUCATION_TYPE_sum_OWN_CAR_AGE,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_REQ_CREDIT_BUREAU_YEAR,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_YEARS_BUILD_AVG,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_ELEVATORS_AVG,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_REGION_POPULATION_RELATIVE,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN,CODE_GENDER_NAME_EDUCATION_TYPE_mean_AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_external_sources_mean,OCCUPATION_TYPE_mean_DAYS_BIRTH,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_APARTMENTS_AVG,OCCUPATION_TYPE_mean_CNT_CHILDREN,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY,OCCUPATION_TYPE_mean_DAYS_REGISTRATION,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH,CODE_GENDER_NAME_EDUCATION_TYPE_mean_external_sources_mean,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_YEARS_BUILD_AVG_abs_diff,OCCUPATION_TYPE_mean_AMT_ANNUITY_abs_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_APARTMENTS_AVG_abs_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_REQ_CREDIT_BUREAU_YEAR_abs_diff,OCCUPATION_TYPE_mean_external_sources_mean_abs_diff,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_external_sources_mean_abs_diff,CODE_GENDER_NAME_EDUCATION_TYPE_mean_AMT_INCOME_TOTAL_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_BASEMENTAREA_AVG_diff,OCCUPATION_TYPE_mean_CNT_FAM_MEMBERS_diff,OCCUPATION_TYPE_mean_CNT_FAM_MEMBERS_abs_diff,OCCUPATION_TYPE_mean_external_sources_mean_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_BASEMENTAREA_AVG_abs_diff,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_REGION_POPULATION_RELATIVE_diff,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT_abs_diff,OCCUPATION_TYPE_mean_DAYS_ID_PUBLISH_diff,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH_diff,OCCUPATION_TYPE_mean_DAYS_EMPLOYED_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_NONLIVINGAREA_AVG_diff,OCCUPATION_TYPE_mean_AMT_ANNUITY_diff,CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_REGION_POPULATION_RELATIVE_abs_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_external_sources_mean_abs_diff,CODE_GENDER_NAME_EDUCATION_TYPE_mean_external_sources_mean_diff,OCCUPATION_TYPE_mean_DAYS_BIRTH_abs_diff,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_ANNUITY_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_NONLIVINGAREA_AVG_abs_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_ELEVATORS_AVG_diff,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_CAR_AGE_diff,OCCUPATION_TYPE_mean_DAYS_ID_PUBLISH_abs_diff,CODE_GENDER_NAME_EDUCATION_TYPE_mean_external_sources_mean_abs_diff,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN_diff,OCCUPATION_TYPE_mean_CNT_CHILDREN_diff,OCCUPATION_TYPE_mean_DAYS_REGISTRATION_diff,OCCUPATION_

In [38]:
df = pd.concat([app_train_new_features, app_test_new_features])
df.to_csv(os.path.join(PATH_TO_DATA, 'application_diff_agg_features.csv'), index=False)

### Bureau

In [4]:
bureau = pd.read_csv(os.path.join(PATH_TO_DATA, 'bureau.csv'))

In [5]:
bureau.head(3)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN


In [6]:
# Simple features
bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)

In [7]:
bureau_new_features = pd.DataFrame({'SK_ID_CURR': bureau['SK_ID_CURR'].unique()})

In [8]:
# Let's make simple aggregations!
BUREAU_AGGREGATION_RECIPIES = [
    ('DAYS_CREDIT', 'count', 'bureau_number_of_past_loans'),
    ('CREDIT_TYPE', 'nunique', 'bureau_number_of_loan_types'),
    ('bureau_credit_active_binary', 'sum', 'bureau_credit_active_sum'), # my
    ('bureau_credit_active_binary', 'mean', 'bureau_credit_active_perc'),
    ('AMT_CREDIT_MAX_OVERDUE', 'sum', 'bureau_sum_of_post_overdue_credits'), # my
    ('AMT_CREDIT_SUM_DEBT', 'sum', 'bureau_total_customer_debt'),
    ('AMT_CREDIT_SUM', 'sum', 'bureau_total_customer_credit'),
    ('AMT_CREDIT_SUM_OVERDUE', 'sum', 'bureau_total_customer_overdue'),
    ('CNT_CREDIT_PROLONG', 'sum', 'bureau_avg_creditdays_prolonged'),
    ('AMT_ANNUITY', 'sum', 'bureau_total_customer_annuity'), # my
    ('bureau_credit_enddate_binary', 'mean', 'bureau_credit_enddate_perc')
]

In [9]:
bureau_new_features = only_sk_id_aggregated_features(bureau, bureau_new_features, BUREAU_AGGREGATION_RECIPIES)

In [10]:
bureau_new_features['bureau_avg_of_past_loans_per_type'] = \
    bureau_new_features['bureau_number_of_past_loans'] / bureau_new_features['bureau_number_of_loan_types']
    
bureau_new_features['bureau_debt_credit_ratio'] = \
    bureau_new_features['bureau_total_customer_debt'] / bureau_new_features['bureau_total_customer_credit']
    
bureau_new_features['bureau_overdue_debt_ratio'] = \
    bureau_new_features['bureau_total_customer_overdue'] / bureau_new_features['bureau_total_customer_debt']
    
bureau_new_features['bureau_overdue_credit_ratio'] = \
    bureau_new_features['bureau_total_customer_overdue'] / bureau_new_features['bureau_total_customer_credit'] # my
    
bureau_new_features['bureau_payment_rate'] = \
    bureau_new_features['bureau_total_customer_annuity'] / bureau_new_features['bureau_total_customer_credit'] # my

In [11]:
# т.к. они повторяются в агрегациях
bureau_new_features.drop([
    'bureau_sum_of_post_overdue_credits',
    'bureau_total_customer_debt',
    'bureau_total_customer_credit',
    'bureau_total_customer_overdue',
    'bureau_avg_creditdays_prolonged',
    'bureau_total_customer_annuity'
], axis=1, inplace=True)

In [12]:
bureau_new_features.head(3)

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_credit_active_sum,bureau_credit_active_perc,bureau_credit_enddate_perc,bureau_avg_of_past_loans_per_type,bureau_debt_credit_ratio,bureau_overdue_debt_ratio,bureau_overdue_credit_ratio,bureau_payment_rate
0,215354,11,3,6,0.545455,0.454545,3.666667,0.047617,0.0,0.0,0.0
1,162297,6,3,3,0.500000,0.166667,2.000000,0.000000,NaN,0.0,0.0
2,402440,1,1,1,1.000000,1.000000,1.000000,0.855355,0.0,0.0,0.0


In [13]:
BUREAU_AGGREGATION_RECIPIES = []

for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_ANNUITY',
                   'AMT_CREDIT_SUM',
                   'AMT_CREDIT_SUM_DEBT',
                   'AMT_CREDIT_SUM_LIMIT',
                   'AMT_CREDIT_SUM_OVERDUE',
                   'AMT_CREDIT_MAX_OVERDUE',
                   'CNT_CREDIT_PROLONG',
                   'CREDIT_DAY_OVERDUE',
                   'DAYS_CREDIT',
                   'DAYS_CREDIT_ENDDATE',
                   'DAYS_CREDIT_UPDATE'
                   ]:
        BUREAU_AGGREGATION_RECIPIES.append((select, agg))

In [14]:
bureau_new_features = only_sk_id_aggregated_features(bureau,
                                                     bureau_new_features,
                                                     BUREAU_AGGREGATION_RECIPIES,
                                                     col_names=False)

In [15]:
# Saving
bureau_new_features.to_csv(os.path.join(PATH_TO_DATA, 'bureau_agg_features.csv'), index=False)

In [16]:
bureau_new_features.head(3)

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_credit_active_sum,bureau_credit_active_perc,bureau_credit_enddate_perc,bureau_avg_of_past_loans_per_type,bureau_debt_credit_ratio,bureau_overdue_debt_ratio,bureau_overdue_credit_ratio,bureau_payment_rate,by_ID_mean_AMT_ANNUITY,by_ID_mean_AMT_CREDIT_SUM,by_ID_mean_AMT_CREDIT_SUM_DEBT,by_ID_mean_AMT_CREDIT_SUM_LIMIT,by_ID_mean_AMT_CREDIT_SUM_OVERDUE,by_ID_mean_AMT_CREDIT_MAX_OVERDUE,by_ID_mean_CNT_CREDIT_PROLONG,by_ID_mean_CREDIT_DAY_OVERDUE,by_ID_mean_DAYS_CREDIT,by_ID_mean_DAYS_CREDIT_ENDDATE,by_ID_mean_DAYS_CREDIT_UPDATE,by_ID_min_AMT_ANNUITY,by_ID_min_AMT_CREDIT_SUM,by_ID_min_AMT_CREDIT_SUM_DEBT,by_ID_min_AMT_CREDIT_SUM_LIMIT,by_ID_min_AMT_CREDIT_SUM_OVERDUE,by_ID_min_AMT_CREDIT_MAX_OVERDUE,by_ID_min_CNT_CREDIT_PROLONG,by_ID_min_CREDIT_DAY_OVERDUE,by_ID_min_DAYS_CREDIT,by_ID_min_DAYS_CREDIT_ENDDATE,by_ID_min_DAYS_CREDIT_UPDATE,by_ID_max_AMT_ANNUITY,by_ID_max_AMT_CREDIT_SUM,by_ID_max_AMT_CREDIT_SUM_DEBT,by_ID_max_AMT_CREDIT_SUM_LIMIT,by_ID_max_AMT_CREDIT_SUM_OVERDUE,by_ID_max_AMT_CREDIT_MAX_OVERDUE,by_ID_max_CNT_CREDIT_PROLONG,by_ID_max_CREDIT_DAY_OVERDUE,by_ID_max_DAYS_CREDIT,by_ID_max_DAYS_CREDIT_ENDDATE,by_ID_max_DAYS_CREDIT_UPDATE,by_ID_sum_AMT_ANNUITY,by_ID_sum_AMT_CREDIT_SUM,by_ID_sum_AMT_CREDIT_SUM_DEBT,by_ID_sum_AMT_CREDIT_SUM_LIMIT,by_ID_sum_AMT_CREDIT_SUM_OVERDUE,by_ID_sum_AMT_CREDIT_MAX_OVERDUE,by_ID_sum_CNT_CREDIT_PROLONG,by_ID_sum_CREDIT_DAY_OVERDUE,by_ID_sum_DAYS_CREDIT,by_ID_sum_DAYS_CREDIT_ENDDATE,by_ID_sum_DAYS_CREDIT_UPDATE,by_ID_var_AMT_ANNUITY,by_ID_var_AMT_CREDIT_SUM,by_ID_var_AMT_CREDIT_SUM_DEBT,by_ID_var_AMT_CREDIT_SUM_LIMIT,by_ID_var_AMT_CREDIT_SUM_OVERDUE,by_ID_var_AMT_CREDIT_MAX_OVERDUE,by_ID_var_CNT_CREDIT_PROLONG,by_ID_var_CREDIT_DAY_OVERDUE,by_ID_var_DAYS_CREDIT,by_ID_var_DAYS_CREDIT_ENDDATE,by_ID_var_DAYS_CREDIT_UPDATE
0,215354,11,3,6,0.545455,0.454545,3.666667,0.047617,0.0,0.0,0.0,NaN,5.430859e+05,47410.53,36327.54,0.0,25891.5,0.0,0.0,-727.818182,2685.3,-367.272727,NaN,42103.8,0.0,0.0,0.0,0.0,0,0,-1872,-1089.0,-1336,NaN,2700000.0,171342.0,108982.62,0.0,77674.5,0,0,-43,27460.0,-16,0.0,5973945.30,284463.18,108982.62,0.0,77674.5,0,0,-8006,26853.0,-4040,NaN,6.043071e+11,4.537546e+09,3.959070e+09,0.0,2.011109e+09,0.0,0.0,432794.763636,7.641379e+07,236203.418182
1,162297,6,3,3,0.500000,0.166667,2.000000,0.000000,NaN,0.0,0.0,NaN,1.371731e+06,0.00,0.00,0.0,2997.0,0.0,0.0,-1344.500000,245.5,-777.833333,NaN,4500.0,0.0,0.0,0.0,0.0,0,0,-2456,-1684.0,-1710,NaN,7033500.0,0.0,0.00,0.0,14985.0,0,0,-277,5261.0,-31,0.0,8230386.15,0.00,0.00,0.0,14985.0,0,0,-8067,1473.0,-4667,NaN,7.753264e+12,0.000000e+00,0.000000e+00,0.0,4.491004e+07,0.0,0.0,559469.500000,6.294475e+06,290229.766667
2,402440,1,1,1,1.000000,1.000000,1.000000,0.855355,0.0,0.0,0.0,NaN,8.991000e+04,76905.00,0.00,0.0,0.0,0.0,0.0,-96.000000,269.0,-22.000000,NaN,89910.0,76905.0,0.0,0.0,0.0,0,0,-96,269.0,-22,NaN,89910.0,76905.0,0.00,0.0,0.0,0,0,-96,269.0,-22,0.0,89910.00,76905.00,0.00,0.0,0.0,0,0,-96,269.0,-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Credit card balance

In [4]:
ccb = pd.read_csv(os.path.join(PATH_TO_DATA, 'credit_card_balance.csv'))

In [5]:
ccb.head(3)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0


In [6]:
# Preprocessing
ccb['AMT_BALANCE'][ccb['AMT_BALANCE'] < 0] = np.nan # my
ccb['AMT_DRAWINGS_ATM_CURRENT'][ccb['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
ccb['AMT_DRAWINGS_CURRENT'][ccb['AMT_DRAWINGS_CURRENT'] < 0] = np.nan
ccb['AMT_RECEIVABLE_PRINCIPAL'][ccb['AMT_RECEIVABLE_PRINCIPAL'] < 0] = np.nan # my
ccb['AMT_RECIVABLE'][ccb['AMT_RECIVABLE'] < 0] = np.nan # my
ccb['AMT_TOTAL_RECEIVABLE'][ccb['AMT_TOTAL_RECEIVABLE'] < 0] = np.nan # my

In [7]:
# Simple features
ccb['ccb_number_of_instalments'] = ccb.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'] \
                                        .agg('max').reset_index()['CNT_INSTALMENT_MATURE_CUM']
ccb['ccb_max_loading_of_credit_limit'] = ccb.groupby(['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL']) \
                                            .apply(lambda x: x.AMT_BALANCE.max() / x.AMT_CREDIT_LIMIT_ACTUAL.max()) \
                                            .reset_index()[0]
        
# my ideas: 
ccb['ccb_balance_limit_ratio'] = ccb['AMT_BALANCE'] / ccb['AMT_CREDIT_LIMIT_ACTUAL']
ccb['ccb_closed_min_regularity'] = (ccb['AMT_PAYMENT_TOTAL_CURRENT'] >= ccb['AMT_INST_MIN_REGULARITY']).astype(int)
ccb['Completed'] = (ccb['NAME_CONTRACT_STATUS'] == 'Completed').astype(int)
ccb['ccb_completed_contracts'] = ccb.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['Completed'] \
                                    .agg('mean').reset_index()['Completed']
ccb['Active'] = (ccb['NAME_CONTRACT_STATUS'] == 'Active').astype(int)
ccb['ccb_active_contracts'] = ccb.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['Active'] \
                                    .agg('mean').reset_index()['Active']

In [8]:
ccb_new_features = pd.DataFrame({'SK_ID_CURR': ccb['SK_ID_CURR'].unique()})

In [9]:
# Let's make simple aggregations!
CCB_AGGREGATION_RECIPIES = [
    ('SK_ID_PREV', 'nunique', 'ccb_number_of_loans'),
    ('ccb_number_of_instalments', 'sum', 'ccb_total_instalments'),
    ('ccb_max_loading_of_credit_limit', 'mean', 'ccb_avg_loading_of_credit_limit'),
    ('AMT_DRAWINGS_ATM_CURRENT', 'sum', 'ccb_drawings_atm'),
    ('AMT_DRAWINGS_CURRENT', 'sum', 'ccb_drawings_total'),
    # my ideas:
    ('ccb_balance_limit_ratio', 'sum', 'ccb_balance_limit_ratio_sum'),
    ('ccb_balance_limit_ratio', 'mean', 'ccb_balance_limit_ratio_mean'),
    ('ccb_closed_min_regularity', 'mean', 'ccb_closed_min_regularity_perc'),
    ('ccb_completed_contracts', 'mean', 'ccb_completed_contracts_perc'),
    ('ccb_active_contracts', 'mean', 'ccb_active_contracts_perc')
]

In [10]:
ccb_new_features = only_sk_id_aggregated_features(ccb,
                                                  ccb_new_features,
                                                  CCB_AGGREGATION_RECIPIES)

In [11]:
ccb_new_features['ccb_cash_card_ratio'] = \
                ccb_new_features['ccb_drawings_atm'] / ccb_new_features['ccb_drawings_total']
ccb_new_features['ccb_installments_per_loan'] = \
                ccb_new_features['ccb_total_instalments'] / ccb_new_features['ccb_number_of_loans']

In [12]:
# т.к. они повторяются в следующих агрегациях
ccb_new_features.drop([
    'ccb_drawings_atm',
    'ccb_drawings_total'
], axis=1, inplace=True)

In [18]:
ccb_new_features.head(3)

,SK_ID_CURR,ccb_number_of_loans,ccb_total_instalments,ccb_avg_loading_of_credit_limit,ccb_balance_limit_ratio_sum,ccb_balance_limit_ratio_mean,ccb_closed_min_regularity_perc,ccb_completed_contracts_perc,ccb_active_contracts_perc,ccb_cash_card_ratio,ccb_installments_per_loan
0,378907,1,0.0,0.696487,6.201723,0.131952,0.957447,0.0,1.0,0.540252,0.0
1,363914,1,94.0,0.202349,51.199173,0.533325,0.968750,0.0,1.0,0.820092,94.0
2,371185,1,63.0,0.525000,13.499703,0.364857,0.972973,0.0,1.0,0.723662,63.0


In [19]:
CCB_AGGREGATION_RECIPIES = []

for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_BALANCE',
                   'AMT_CREDIT_LIMIT_ACTUAL',
                   'AMT_DRAWINGS_ATM_CURRENT',
                   'AMT_DRAWINGS_CURRENT',
                   'AMT_DRAWINGS_OTHER_CURRENT',
                   'AMT_DRAWINGS_POS_CURRENT',
                   'AMT_PAYMENT_TOTAL_CURRENT',
                   'CNT_DRAWINGS_ATM_CURRENT',
                   'CNT_DRAWINGS_CURRENT',
                   'CNT_DRAWINGS_OTHER_CURRENT',
                   'CNT_INSTALMENT_MATURE_CUM',
                   'SK_DPD',
                   'SK_DPD_DEF'
                   ]:
        CCB_AGGREGATION_RECIPIES.append((select, agg))

In [20]:
ccb_new_features = only_sk_id_aggregated_features(ccb,
                                                  ccb_new_features,
                                                  CCB_AGGREGATION_RECIPIES,
                                                  col_names=False)

In [21]:
ccb_new_features.head(3)

,SK_ID_CURR,ccb_number_of_loans,ccb_total_instalments,ccb_avg_loading_of_credit_limit,ccb_balance_limit_ratio_sum,ccb_balance_limit_ratio_mean,ccb_closed_min_regularity_perc,ccb_completed_contracts_perc,ccb_active_contracts_perc,ccb_cash_card_ratio,ccb_installments_per_loan,by_ID_mean_AMT_BALANCE,by_ID_mean_AMT_CREDIT_LIMIT_ACTUAL,by_ID_mean_AMT_DRAWINGS_ATM_CURRENT,by_ID_mean_AMT_DRAWINGS_CURRENT,by_ID_mean_AMT_DRAWINGS_OTHER_CURRENT,by_ID_mean_AMT_DRAWINGS_POS_CURRENT,by_ID_mean_AMT_PAYMENT_TOTAL_CURRENT,by_ID_mean_CNT_DRAWINGS_ATM_CURRENT,by_ID_mean_CNT_DRAWINGS_CURRENT,by_ID_mean_CNT_DRAWINGS_OTHER_CURRENT,by_ID_mean_CNT_INSTALMENT_MATURE_CUM,by_ID_mean_SK_DPD,by_ID_mean_SK_DPD_DEF,by_ID_min_AMT_BALANCE,by_ID_min_AMT_CREDIT_LIMIT_ACTUAL,by_ID_min_AMT_DRAWINGS_ATM_CURRENT,by_ID_min_AMT_DRAWINGS_CURRENT,by_ID_min_AMT_DRAWINGS_OTHER_CURRENT,by_ID_min_AMT_DRAWINGS_POS_CURRENT,by_ID_min_AMT_PAYMENT_TOTAL_CURRENT,by_ID_min_CNT_DRAWINGS_ATM_CURRENT,by_ID_min_CNT_DRAWINGS_CURRENT,by_ID_min_CNT_DRAWINGS_OTHER_CURRENT,by_ID_min_CNT_INSTALMENT_MATURE_CUM,by_ID_min_SK_DPD,by_ID_min_SK_DPD_DEF,by_ID_max_AMT_BALANCE,by_ID_max_AMT_CREDIT_LIMIT_ACTUAL,by_ID_max_AMT_DRAWINGS_ATM_CURRENT,by_ID_max_AMT_DRAWINGS_CURRENT,by_ID_max_AMT_DRAWINGS_OTHER_CURRENT,by_ID_max_AMT_DRAWINGS_POS_CURRENT,by_ID_max_AMT_PAYMENT_TOTAL_CURRENT,by_ID_max_CNT_DRAWINGS_ATM_CURRENT,by_ID_max_CNT_DRAWINGS_CURRENT,by_ID_max_CNT_DRAWINGS_OTHER_CURRENT,by_ID_max_CNT_INSTALMENT_MATURE_CUM,by_ID_max_SK_DPD,by_ID_max_SK_DPD_DEF,by_ID_sum_AMT_BALANCE,by_ID_sum_AMT_CREDIT_LIMIT_ACTUAL,by_ID_sum_AMT_DRAWINGS_ATM_CURRENT,by_ID_sum_AMT_DRAWINGS_CURRENT,by_ID_sum_AMT_DRAWINGS_OTHER_CURRENT,by_ID_sum_AMT_DRAWINGS_POS_CURRENT,by_ID_sum_AMT_PAYMENT_TOTAL_CURRENT,by_ID_sum_CNT_DRAWINGS_ATM_CURRENT,by_ID_sum_CNT_DRAWINGS_CURRENT,by_ID_sum_CNT_DRAWINGS_OTHER_CURRENT,by_ID_sum_CNT_INSTALMENT_MATURE_CUM,by_ID_sum_SK_DPD,by_ID_sum_SK_DPD_DEF,by_ID_var_AMT_BALANCE,by_ID_var_AMT_CREDIT_LIMIT_ACTUAL,by_ID_var_AMT_DRAWINGS_ATM_CURRENT,by_ID_var_AMT_DRAWINGS_CURRENT,by_ID_var_AMT_DRAWINGS_OTHER_CURRENT,by_ID_var_AMT_DRAWINGS_POS_CURRENT,by_ID_var_AMT_PAYMENT_TOTAL_CURRENT,by_ID_var_CNT_DRAWINGS_ATM_CURRENT,by_ID_var_CNT_DRAWINGS_CURRENT,by_ID_var_CNT_DRAWINGS_OTHER_CURRENT,by_ID_var_CNT_INSTALMENT_MATURE_CUM,by_ID_var_SK_DPD,by_ID_var_SK_DPD_DEF
0,378907,1,0.0,0.696487,6.201723,0.131952,0.957447,0.0,1.0,0.540252,0.0,27973.440000,192255.319149,1436.170213,2658.332872,0.0,1222.162660,3835.946489,0.021277,0.170213,0.0,21.369565,0.127660,0.127660,0.0,9000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0,0,69186.69,270000,67500.0,67500.0,0.0,26640.000,31500.0,1.0,2,0.0,39.0,6,6,1314751.680,9036000,67500.0,124941.645,0.0,57441.645,180289.485,1.0,8,0.0,983.0,6,6,6.081844e+08,1.092946e+10,9.694149e+07,1.184815e+08,0.0,2.512682e+07,3.425500e+07,0.021277,0.187789,0.0,120.460386,0.765957,0.765957
1,363914,1,94.0,0.202349,51.199173,0.533325,0.968750,0.0,1.0,0.820092,94.0,32895.901875,63750.000000,3060.937500,3732.434063,0.0,671.496562,5061.562500,0.322917,0.437500,0.0,43.843750,0.010417,0.010417,0.0,45000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,4.0,0,0,72964.44,67500,62100.0,62100.0,0.0,32563.755,49500.0,6.0,7,0.0,69.0,1,1,3158006.580,6120000,293850.0,358313.670,0.0,64463.670,485910.000,31.0,42,0.0,4209.0,1,1,7.080661e+08,7.105263e+07,1.101541e+08,1.426229e+08,0.0,2.141806e+07,6.420004e+07,0.915680,1.806579,0.0,372.027961,0.010417,0.010417
2,371185,1,63.0,0.525000,13.499703,0.364857,0.972973,0.0,1.0,0.723662,63.0,112572.811216,378364.864865,14062.500000,18907.208514,0.0,5369.908750,19976.959459,0.583333,2.810811,0.0,18.500000,0.000000,0.000000,0.0,112500,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0,0,273194.64,450000,119250.0,142296.3,0.0,113969.430,283950.0,6.0,44,0.0,36.0,0,0,4165194.015,13999500,506250.0,699566.715,0.0,193316.715,739147.500,21.0,104,0.0,666.0,0,0,4.750472e+09,1.917830e+10,6.426121e+08,1.075961e+09,0.0,3.710275e+08,3.573750e+09,1.164286,56.768769,0.0,111.000000,0.000000,0.000000


In [22]:
# Merge
ccb_new_features.to_csv(os.path.join(PATH_TO_DATA, 'credit_card_agg_features.csv'), index=False)

### Installments

In [42]:
installments = pd.read_csv(os.path.join(PATH_TO_DATA, 'installments_payments.csv'))

In [43]:
installments.head(3)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000


In [44]:
INSTALLMENTS_AGGREGATION_RECIPIES = []

for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_INSTALMENT',
                   'AMT_PAYMENT',
                   'DAYS_ENTRY_PAYMENT',
                   'DAYS_INSTALMENT',
                   'NUM_INSTALMENT_NUMBER'
                   ]:
        INSTALLMENTS_AGGREGATION_RECIPIES.append((select, agg))

In [45]:
installments_new_features = pd.DataFrame({'SK_ID_CURR': installments['SK_ID_CURR'].unique()})
installments_new_features = only_sk_id_aggregated_features(installments,
                                                           installments_new_features,
                                                           INSTALLMENTS_AGGREGATION_RECIPIES,
                                                           col_names=False)

In [46]:
installments_new_features.head(3)

,SK_ID_CURR,by_ID_mean_AMT_INSTALMENT,by_ID_mean_AMT_PAYMENT,by_ID_mean_DAYS_ENTRY_PAYMENT,by_ID_mean_DAYS_INSTALMENT,by_ID_mean_NUM_INSTALMENT_NUMBER,by_ID_min_AMT_INSTALMENT,by_ID_min_AMT_PAYMENT,by_ID_min_DAYS_ENTRY_PAYMENT,by_ID_min_DAYS_INSTALMENT,by_ID_min_NUM_INSTALMENT_NUMBER,by_ID_max_AMT_INSTALMENT,by_ID_max_AMT_PAYMENT,by_ID_max_DAYS_ENTRY_PAYMENT,by_ID_max_DAYS_INSTALMENT,by_ID_max_NUM_INSTALMENT_NUMBER,by_ID_sum_AMT_INSTALMENT,by_ID_sum_AMT_PAYMENT,by_ID_sum_DAYS_ENTRY_PAYMENT,by_ID_sum_DAYS_INSTALMENT,by_ID_sum_NUM_INSTALMENT_NUMBER,by_ID_var_AMT_INSTALMENT,by_ID_var_AMT_PAYMENT,by_ID_var_DAYS_ENTRY_PAYMENT,by_ID_var_DAYS_INSTALMENT,by_ID_var_NUM_INSTALMENT_NUMBER
0,161674,12600.013812,12600.013812,-1037.544554,-1026.643564,10.207921,3647.700,3647.700,-2926.0,-2899.0,1,117414.000,117414.000,-62.0,-60.0,36,1272601.395,1272601.395,-104792.0,-103691.0,1031,1.742657e+08,1.742657e+08,580652.510495,583596.271683,83.706337
1,151639,10027.751582,9240.438418,-1330.822785,-1327.765823,37.784810,354.870,5.040,-2926.0,-2919.0,1,205878.825,205878.825,-13.0,-13.0,104,1584384.750,1459989.270,-210270.0,-209787.0,5970,3.541087e+08,3.502881e+08,786577.598968,783419.632710,1033.469322
2,193053,11483.070000,11483.070000,-35.000000,-40.333333,1.333333,631.035,631.035,-63.0,-63.0,1,25425.000,25425.000,-21.0,-14.0,2,34449.210,34449.210,-105.0,-121.0,4,1.608458e+08,1.608458e+08,588.000000,610.333333,0.333333


In [47]:
# Simple features
installments['inst_paid_late_in_days'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']
installments['inst_paid_late'] = (installments['inst_paid_late_in_days'] > 0).astype(int)
installments['inst_paid_over_amount'] = installments['AMT_PAYMENT'] - installments['AMT_INSTALMENT']
installments['inst_paid_over'] = (installments['inst_paid_over_amount'] > 0).astype(int)

In [48]:
INSTALLMENTS_AGGREGATION_RECIPIES = []

for agg in ['mean', 'median', 'min', 'max', 'sum', 'std', skew, kurtosis, iqr]:
    for select in ['NUM_INSTALMENT_VERSION',
                   'inst_paid_late_in_days',
                   'inst_paid_over_amount'
                   ]:
        INSTALLMENTS_AGGREGATION_RECIPIES.append((select, agg))
        
for agg in ['mean', 'sum']:
    for select in ['inst_paid_late',
                   'inst_paid_over'
                  ]:
        INSTALLMENTS_AGGREGATION_RECIPIES.append((select, agg))

In [49]:
installments_new_features = only_sk_id_aggregated_features(installments,
                                                           installments_new_features,
                                                           INSTALLMENTS_AGGREGATION_RECIPIES,
                                                           col_names=False)

In [51]:
installments_new_features.rename(index=str, inplace=True, columns={
    'by_ID_<function skew at 0x000002A1A9D61378>_NUM_INSTALMENT_VERSION': 'by_ID_skew_NUM_INSTALMENT_VERSION',
    'by_ID_<function skew at 0x000002A1A9D61378>_inst_paid_late_in_days': 'by_ID_skew_inst_paid_late_in_days',
    'by_ID_<function skew at 0x000002A1A9D61378>_inst_paid_over_amount': 'by_ID_skew_inst_paid_over_amount',
    'by_ID_<function kurtosis at 0x000002A1A9D61400>_NUM_INSTALMENT_VERSION': 'by_ID_kurtosis_NUM_INSTALMENT_VERSION',
    'by_ID_<function kurtosis at 0x000002A1A9D61400>_inst_paid_late_in_days': 'by_ID_kurtosis_inst_paid_late_in_days',
    'by_ID_<function kurtosis at 0x000002A1A9D61400>_inst_paid_over_amount': 'by_ID_kurtosis_inst_paid_over_amount',
    'by_ID_<function iqr at 0x000002A1A9D9BA60>_NUM_INSTALMENT_VERSION': 'by_ID_iqr_NUM_INSTALMENT_VERSION',
    'by_ID_<function iqr at 0x000002A1A9D9BA60>_inst_paid_late_in_days': 'by_ID_iqr_inst_paid_late_in_days',
    'by_ID_<function iqr at 0x000002A1A9D9BA60>_inst_paid_over_amount': 'by_ID_iqr_inst_paid_over_amount'
})

In [52]:
installments_new_features.head(3)

,SK_ID_CURR,by_ID_mean_AMT_INSTALMENT,by_ID_mean_AMT_PAYMENT,by_ID_mean_DAYS_ENTRY_PAYMENT,by_ID_mean_DAYS_INSTALMENT,by_ID_mean_NUM_INSTALMENT_NUMBER,by_ID_min_AMT_INSTALMENT,by_ID_min_AMT_PAYMENT,by_ID_min_DAYS_ENTRY_PAYMENT,by_ID_min_DAYS_INSTALMENT,by_ID_min_NUM_INSTALMENT_NUMBER,by_ID_max_AMT_INSTALMENT,by_ID_max_AMT_PAYMENT,by_ID_max_DAYS_ENTRY_PAYMENT,by_ID_max_DAYS_INSTALMENT,by_ID_max_NUM_INSTALMENT_NUMBER,by_ID_sum_AMT_INSTALMENT,by_ID_sum_AMT_PAYMENT,by_ID_sum_DAYS_ENTRY_PAYMENT,by_ID_sum_DAYS_INSTALMENT,by_ID_sum_NUM_INSTALMENT_NUMBER,by_ID_var_AMT_INSTALMENT,by_ID_var_AMT_PAYMENT,by_ID_var_DAYS_ENTRY_PAYMENT,by_ID_var_DAYS_INSTALMENT,by_ID_var_NUM_INSTALMENT_NUMBER,by_ID_mean_NUM_INSTALMENT_VERSION,by_ID_mean_inst_paid_late_in_days,by_ID_mean_inst_paid_over_amount,by_ID_median_NUM_INSTALMENT_VERSION,by_ID_median_inst_paid_late_in_days,by_ID_median_inst_paid_over_amount,by_ID_min_NUM_INSTALMENT_VERSION,by_ID_min_inst_paid_late_in_days,by_ID_min_inst_paid_over_amount,by_ID_max_NUM_INSTALMENT_VERSION,by_ID_max_inst_paid_late_in_days,by_ID_max_inst_paid_over_amount,by_ID_sum_NUM_INSTALMENT_VERSION,by_ID_sum_inst_paid_late_in_days,by_ID_sum_inst_paid_over_amount,by_ID_std_NUM_INSTALMENT_VERSION,by_ID_std_inst_paid_late_in_days,by_ID_std_inst_paid_over_amount,by_ID_skew_NUM_INSTALMENT_VERSION,by_ID_skew_inst_paid_late_in_days,by_ID_skew_inst_paid_over_amount,by_ID_kurtosis_NUM_INSTALMENT_VERSION,by_ID_kurtosis_inst_paid_late_in_days,by_ID_kurtosis_inst_paid_over_amount,by_ID_iqr_NUM_INSTALMENT_VERSION,by_ID_iqr_inst_paid_late_in_days,by_ID_iqr_inst_paid_over_amount,by_ID_mean_inst_paid_late,by_ID_mean_inst_paid_over,by_ID_sum_inst_paid_late,by_ID_sum_inst_paid_over
0,161674,12600.013812,12600.013812,-1037.544554,-1026.643564,10.207921,3647.700,3647.700,-2926.0,-2899.0,1,117414.000,117414.000,-62.0,-60.0,36,1272601.395,1272601.395,-104792.0,-103691.0,1031,1.742657e+08,1.742657e+08,580652.510495,583596.271683,83.706337,1.039604,-10.900990,0.000000,1.0,-6.0,0.0,1.0,-65.0,0.000,2.0,0.0,0.0,105.0,-1101.0,0.00,0.196000,13.872639,0.000000,4.721360,-2.155903,0.000000,20.291237,4.684767,-3.000000,0.0,12.0,0.0,0.000000,0.0,0,0
1,151639,10027.751582,9240.438418,-1330.822785,-1327.765823,37.784810,354.870,5.040,-2926.0,-2919.0,1,205878.825,205878.825,-13.0,-13.0,104,1584384.750,1459989.270,-210270.0,-209787.0,5970,3.541087e+08,3.502881e+08,786577.598968,783419.632710,1033.469322,0.360759,-3.056962,-787.313165,0.0,0.0,0.0,0.0,-38.0,-26067.465,2.0,9.0,0.0,57.0,-483.0,-124395.48,0.507502,7.669315,4001.769578,0.871002,-2.672848,-5.590484,-0.580850,7.036910,30.435489,1.0,2.0,0.0,0.069620,0.0,11,0
2,193053,11483.070000,11483.070000,-35.000000,-40.333333,1.333333,631.035,631.035,-63.0,-63.0,1,25425.000,25425.000,-21.0,-14.0,2,34449.210,34449.210,-105.0,-121.0,4,1.608458e+08,1.608458e+08,588.000000,610.333333,0.333333,2.666667,5.333333,0.000000,3.0,0.0,0.0,2.0,-7.0,0.000,3.0,23.0,0.0,8.0,16.0,0.00,0.577350,15.695010,0.000000,-0.707107,0.552187,0.000000,-1.500000,-1.500000,-3.000000,0.5,15.0,0.0,0.333333,0.0,1,0


In [53]:
# Saving
installments_new_features.to_csv(os.path.join(PATH_TO_DATA, 'installments_agg_features.csv'), index=False)

### Pos_cash_balance

In [17]:
pos_cash = pd.read_csv(os.path.join(PATH_TO_DATA, 'POS_CASH_balance.csv'))

In [18]:
pos_cash.head(3)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0


In [19]:
POS_CASH_AGGREGATION_RECIPIES = []

for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['MONTHS_BALANCE',
                   'SK_DPD',
                   'SK_DPD_DEF'
                  ]:
        POS_CASH_AGGREGATION_RECIPIES.append((select, agg))

In [20]:
pos_cash_new_features = pd.DataFrame({'SK_ID_CURR': pos_cash['SK_ID_CURR'].unique()})
pos_cash_new_features = only_sk_id_aggregated_features(pos_cash,
                                                       pos_cash_new_features,
                                                       POS_CASH_AGGREGATION_RECIPIES,
                                                       col_names=False)

In [21]:
pos_cash_new_features.head(3)

,SK_ID_CURR,by_ID_mean_MONTHS_BALANCE,by_ID_mean_SK_DPD,by_ID_mean_SK_DPD_DEF,by_ID_min_MONTHS_BALANCE,by_ID_min_SK_DPD,by_ID_min_SK_DPD_DEF,by_ID_max_MONTHS_BALANCE,by_ID_max_SK_DPD,by_ID_max_SK_DPD_DEF,by_ID_sum_MONTHS_BALANCE,by_ID_sum_SK_DPD,by_ID_sum_SK_DPD_DEF,by_ID_var_MONTHS_BALANCE,by_ID_var_SK_DPD,by_ID_var_SK_DPD_DEF
0,182943,-27.186047,0.000000,0.000000,-62,0,0,-2,0,0,-1169,0,0,358.297896,0.000000,0.000000
1,367990,-41.148148,0.000000,0.000000,-83,0,0,-16,0,0,-1111,0,0,666.669516,0.000000,0.000000
2,397406,-42.449541,37.706422,0.146789,-94,0,0,-1,485,6,-4627,4110,16,751.638634,11194.024125,0.793068


**Hand crafted features**

In [22]:
pos_cash_sorted = pos_cash.sort_values(['SK_ID_CURR', 'MONTHS_BALANCE'])

In [24]:
# начисления, оставленные для оплаты предыдущего кредита
group_object = pos_cash_sorted.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].last().reset_index()
group_object.rename(index=str, columns={'CNT_INSTALMENT_FUTURE': 'pos_cash_remaining_installments'}, inplace=True)
pos_cash_new_features = pos_cash_new_features.merge(group_object, on='SK_ID_CURR', how='left')

In [25]:
# сколько у клиента заверешенных конрактов относительно всего кол-ва
pos_cash['is_contract_status_completed'] = (pos_cash['NAME_CONTRACT_STATUS'] == 'Completed').astype(int)
group_object = pos_cash.groupby('SK_ID_CURR')['is_contract_status_completed'].sum().reset_index()
group_object.rename(index=str, columns={'is_contract_status_completed': 'pos_cash_completed_contracts'}, inplace=True)
pos_cash_new_features = pos_cash_new_features.merge(group_object, on='SK_ID_CURR', how='left')

group_object = pos_cash.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique().reset_index()
group_object.rename(index=str, columns={'SK_ID_PREV': 'pos_cash_counts_of_contracts'}, inplace=True)
pos_cash_new_features = pos_cash_new_features.merge(group_object, on='SK_ID_CURR', how='left')

pos_cash_new_features['pos_cash_completed_contracts_ratio'] = \
        pos_cash_new_features['pos_cash_completed_contracts'] / pos_cash_new_features['pos_cash_counts_of_contracts']

In [26]:
pos_cash_new_features.head(3)

,SK_ID_CURR,by_ID_mean_MONTHS_BALANCE,by_ID_mean_SK_DPD,by_ID_mean_SK_DPD_DEF,by_ID_min_MONTHS_BALANCE,by_ID_min_SK_DPD,by_ID_min_SK_DPD_DEF,by_ID_max_MONTHS_BALANCE,by_ID_max_SK_DPD,by_ID_max_SK_DPD_DEF,by_ID_sum_MONTHS_BALANCE,by_ID_sum_SK_DPD,by_ID_sum_SK_DPD_DEF,by_ID_var_MONTHS_BALANCE,by_ID_var_SK_DPD,by_ID_var_SK_DPD_DEF,pos_cash_remaining_installments,pos_cash_completed_contracts,pos_cash_counts_of_contracts,pos_cash_completed_contracts_ratio
0,182943,-27.186047,0.000000,0.000000,-62,0,0,-2,0,0,-1169,0,0,358.297896,0.000000,0.000000,15.0,1,2,0.50
1,367990,-41.148148,0.000000,0.000000,-83,0,0,-16,0,0,-1111,0,0,666.669516,0.000000,0.000000,0.0,2,2,1.00
2,397406,-42.449541,37.706422,0.146789,-94,0,0,-1,485,6,-4627,4110,16,751.638634,11194.024125,0.793068,0.0,6,8,0.75


In [27]:
# Saving
pos_cash_new_features.to_csv(os.path.join(PATH_TO_DATA, 'pos_cash_agg_features.csv'), index=False)

### Previous application

In [29]:
prev_app = pd.read_csv(os.path.join(PATH_TO_DATA, 'previous_application.csv'))

In [30]:
prev_app.head(3)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0


In [31]:
# Preprocessing
for col in ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
            'DAYS_LAST_DUE', 'DAYS_TERMINATION']:
    prev_app[col].replace(365243, np.nan, inplace=True)

In [32]:
PREV_APP_AGGREGATION_RECIPIES = []

for agg in ['min', 'max', 'mean', 'sum', 'var']:
    for select in ['AMT_ANNUITY',
                   'AMT_APPLICATION',
                   'AMT_CREDIT',
                   'AMT_DOWN_PAYMENT',
                   'AMT_GOODS_PRICE',
                   'CNT_PAYMENT',
                   'DAYS_DECISION',
                   'DAYS_FIRST_DUE', #my
                   'HOUR_APPR_PROCESS_START',
                   'RATE_DOWN_PAYMENT'
                   ]:
        PREV_APP_AGGREGATION_RECIPIES.append((select, agg))

In [33]:
prev_app_new_features = pd.DataFrame({'SK_ID_CURR': prev_app['SK_ID_CURR'].unique()})
prev_app_new_features = only_sk_id_aggregated_features(prev_app,
                                                       prev_app_new_features,
                                                       PREV_APP_AGGREGATION_RECIPIES,
                                                       col_names=False)

In [34]:
prev_app_new_features.head(3)

,SK_ID_CURR,by_ID_min_AMT_ANNUITY,by_ID_min_AMT_APPLICATION,by_ID_min_AMT_CREDIT,by_ID_min_AMT_DOWN_PAYMENT,by_ID_min_AMT_GOODS_PRICE,by_ID_min_CNT_PAYMENT,by_ID_min_DAYS_DECISION,by_ID_min_DAYS_FIRST_DUE,by_ID_min_HOUR_APPR_PROCESS_START,by_ID_min_RATE_DOWN_PAYMENT,by_ID_max_AMT_ANNUITY,by_ID_max_AMT_APPLICATION,by_ID_max_AMT_CREDIT,by_ID_max_AMT_DOWN_PAYMENT,by_ID_max_AMT_GOODS_PRICE,by_ID_max_CNT_PAYMENT,by_ID_max_DAYS_DECISION,by_ID_max_DAYS_FIRST_DUE,by_ID_max_HOUR_APPR_PROCESS_START,by_ID_max_RATE_DOWN_PAYMENT,by_ID_mean_AMT_ANNUITY,by_ID_mean_AMT_APPLICATION,by_ID_mean_AMT_CREDIT,by_ID_mean_AMT_DOWN_PAYMENT,by_ID_mean_AMT_GOODS_PRICE,by_ID_mean_CNT_PAYMENT,by_ID_mean_DAYS_DECISION,by_ID_mean_DAYS_FIRST_DUE,by_ID_mean_HOUR_APPR_PROCESS_START,by_ID_mean_RATE_DOWN_PAYMENT,by_ID_sum_AMT_ANNUITY,by_ID_sum_AMT_APPLICATION,by_ID_sum_AMT_CREDIT,by_ID_sum_AMT_DOWN_PAYMENT,by_ID_sum_AMT_GOODS_PRICE,by_ID_sum_CNT_PAYMENT,by_ID_sum_DAYS_DECISION,by_ID_sum_DAYS_FIRST_DUE,by_ID_sum_HOUR_APPR_PROCESS_START,by_ID_sum_RATE_DOWN_PAYMENT,by_ID_var_AMT_ANNUITY,by_ID_var_AMT_APPLICATION,by_ID_var_AMT_CREDIT,by_ID_var_AMT_DOWN_PAYMENT,by_ID_var_AMT_GOODS_PRICE,by_ID_var_CNT_PAYMENT,by_ID_var_DAYS_DECISION,by_ID_var_DAYS_FIRST_DUE,by_ID_var_HOUR_APPR_PROCESS_START,by_ID_var_RATE_DOWN_PAYMENT
0,271877,1730.43,17145.0,17145.0,0.0,17145.0,12.0,-548,-512.0,14,0.0,68258.655,1800000.0,1754721.0,180000.0,1800000.0,36.0,-73,-42.0,18,0.101325,27468.825,641848.50,630571.500,60000.00,641848.5,20.000000,-364.333333,-277.000000,15.666667,0.033775,82406.475,1925545.5,1891714.5,180000.0,1925545.5,60.0,-1093,-554.0,47,0.101325,1.276410e+09,1.008068e+12,9.504211e+11,1.080000e+10,1.008068e+12,192.000000,6.510033e+04,1.104500e+05,4.333333,0.003422
1,108129,4830.93,0.0,23688.0,0.0,33052.5,0.0,-1208,-1176.0,9,0.0,25188.615,607500.0,679671.0,24750.0,607500.0,36.0,-164,-134.0,13,0.556485,13039.485,201525.75,257522.250,8250.00,241830.9,15.666667,-713.666667,-639.833333,10.500000,0.185495,78236.910,1209154.5,1545133.5,24750.0,1209154.5,94.0,-4282,-3839.0,63,0.556485,7.075052e+07,6.725932e+10,7.678783e+10,2.041875e+08,7.189037e+10,263.066667,1.330471e+05,1.183198e+05,2.300000,0.103225
2,122040,6300.00,0.0,0.0,0.0,58545.0,0.0,-2576,-2545.0,10,0.0,15060.735,112500.0,136444.5,9571.5,112500.0,12.0,-301,-271.0,11,0.104691,9623.115,66690.00,72696.375,4785.75,88920.0,8.000000,-1395.750000,-1727.000000,10.750000,0.052345,28869.345,266760.0,290785.5,9571.5,266760.0,24.0,-5583,-5181.0,43,0.104691,2.254097e+07,2.484968e+09,3.239271e+09,4.580681e+07,7.624145e+08,48.000000,1.602844e+06,1.598052e+06,0.250000,0.005480


**Hand crafted features**

In [35]:
prev_app_sorted = prev_app.sort_values(['SK_ID_CURR', 'DAYS_DECISION'])

In [36]:
# число предыдущих кредитов
group_object = prev_app_sorted.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique().reset_index()
group_object.rename(index=str, columns={'SK_ID_PREV': 'prev_app_number_of_prev_app'}, inplace=True)
prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')

In [38]:
# 2 бинарных признака: последняя заявка утверждена или отказана
# это не взаимозаменяющие признаки т.к. существуют еще типы: Canceled и Unused offer
prev_app_sorted['prev_app_prev_was_approved'] = (prev_app_sorted['NAME_CONTRACT_STATUS'] == 'Approved').astype(int)
group_object = prev_app_sorted.groupby(['SK_ID_CURR'])['prev_app_prev_was_approved'].last().reset_index()
prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')

prev_app_sorted['prev_app_prev_was_refused'] = (prev_app_sorted['NAME_CONTRACT_STATUS'] == 'Refused').astype(int)
group_object = prev_app_sorted.groupby(['SK_ID_CURR'])['prev_app_prev_was_refused'].last().reset_index()
prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')

# Среднее число утвержденных и закрытых заявок
prev_app_sorted['prev_app_mean_approved_contracts'] = (prev_app_sorted['NAME_CONTRACT_STATUS'] == 'Approved').astype(int)
group_object = prev_app_sorted.groupby(['SK_ID_CURR'])['prev_app_mean_approved_contracts'].mean().reset_index()
prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')

prev_app_sorted['prev_app_mean_canceled_contracts'] = (prev_app_sorted['NAME_CONTRACT_STATUS'] == 'Canceled').astype(int)
group_object = prev_app_sorted.groupby(['SK_ID_CURR'])['prev_app_mean_canceled_contracts'].mean().reset_index()
prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')

In [39]:
numbers_of_applications = [1, 3, 5]

for number in numbers_of_applications:
    prev_app_tail = prev_app_sorted.groupby('SK_ID_CURR').tail(number)
    
    # срок последних 1/3/5 кредитов
    group_object = prev_app_tail.groupby('SK_ID_CURR')['CNT_PAYMENT'].mean().reset_index()
    group_object.rename(index=str,
                        columns={'CNT_PAYMENT': 'prev_app_term_of_last_{}_credits_mean'.format(number)},
                        inplace=True)
    prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')
    
    # дата принятия решения последних 1/3/5 кредитов относительно текущего кредита
    group_object = prev_app_tail.groupby('SK_ID_CURR')['DAYS_DECISION'].mean().reset_index()
    group_object.rename(index=str,
                        columns={'DAYS_DECISION': 'prev_app_days_decision_about_last_{}_credits_mean'.format(number)},
                        inplace=True)
    prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')
    
    # дата первой выплаты последних 1/3/5 кредитов относительно текущего кредита
    group_object = prev_app_tail.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].mean().reset_index()
    group_object.rename(index=str,
                        columns={'DAYS_FIRST_DRAWING': 'prev_app_days_first_drawing_about_last_{}_credits_mean'.format(number)},
                        inplace=True)
    prev_app_new_features = prev_app_new_features.merge(group_object, on='SK_ID_CURR', how='left')

In [40]:
prev_app_new_features.head(3)

,SK_ID_CURR,by_ID_min_AMT_ANNUITY,by_ID_min_AMT_APPLICATION,by_ID_min_AMT_CREDIT,by_ID_min_AMT_DOWN_PAYMENT,by_ID_min_AMT_GOODS_PRICE,by_ID_min_CNT_PAYMENT,by_ID_min_DAYS_DECISION,by_ID_min_DAYS_FIRST_DUE,by_ID_min_HOUR_APPR_PROCESS_START,by_ID_min_RATE_DOWN_PAYMENT,by_ID_max_AMT_ANNUITY,by_ID_max_AMT_APPLICATION,by_ID_max_AMT_CREDIT,by_ID_max_AMT_DOWN_PAYMENT,by_ID_max_AMT_GOODS_PRICE,by_ID_max_CNT_PAYMENT,by_ID_max_DAYS_DECISION,by_ID_max_DAYS_FIRST_DUE,by_ID_max_HOUR_APPR_PROCESS_START,by_ID_max_RATE_DOWN_PAYMENT,by_ID_mean_AMT_ANNUITY,by_ID_mean_AMT_APPLICATION,by_ID_mean_AMT_CREDIT,by_ID_mean_AMT_DOWN_PAYMENT,by_ID_mean_AMT_GOODS_PRICE,by_ID_mean_CNT_PAYMENT,by_ID_mean_DAYS_DECISION,by_ID_mean_DAYS_FIRST_DUE,by_ID_mean_HOUR_APPR_PROCESS_START,by_ID_mean_RATE_DOWN_PAYMENT,by_ID_sum_AMT_ANNUITY,by_ID_sum_AMT_APPLICATION,by_ID_sum_AMT_CREDIT,by_ID_sum_AMT_DOWN_PAYMENT,by_ID_sum_AMT_GOODS_PRICE,by_ID_sum_CNT_PAYMENT,by_ID_sum_DAYS_DECISION,by_ID_sum_DAYS_FIRST_DUE,by_ID_sum_HOUR_APPR_PROCESS_START,by_ID_sum_RATE_DOWN_PAYMENT,by_ID_var_AMT_ANNUITY,by_ID_var_AMT_APPLICATION,by_ID_var_AMT_CREDIT,by_ID_var_AMT_DOWN_PAYMENT,by_ID_var_AMT_GOODS_PRICE,by_ID_var_CNT_PAYMENT,by_ID_var_DAYS_DECISION,by_ID_var_DAYS_FIRST_DUE,by_ID_var_HOUR_APPR_PROCESS_START,by_ID_var_RATE_DOWN_PAYMENT,prev_app_number_of_prev_app,prev_app_prev_was_approved,prev_app_prev_was_refused,prev_app_mean_approved_contracts,prev_app_mean_canceled_contracts,prev_app_term_of_last_1_credits_mean,prev_app_days_decision_about_last_1_credits_mean,prev_app_days_first_drawing_about_last_1_credits_mean,prev_app_term_of_last_3_credits_mean,prev_app_days_decision_about_last_3_credits_mean,prev_app_days_first_drawing_about_last_3_credits_mean,prev_app_term_of_last_5_credits_mean,prev_app_days_decision_about_last_5_credits_mean,prev_app_days_first_drawing_about_last_5_credits_mean
0,271877,1730.43,17145.0,17145.0,0.0,17145.0,12.0,-548,-512.0,14,0.0,68258.655,1800000.0,1754721.0,180000.0,1800000.0,36.0,-73,-42.0,18,0.101325,27468.825,641848.50,630571.500,60000.00,641848.5,20.000000,-364.333333,-277.000000,15.666667,0.033775,82406.475,1925545.5,1891714.5,180000.0,1925545.5,60.0,-1093,-554.0,47,0.101325,1.276410e+09,1.008068e+12,9.504211e+11,1.080000e+10,1.008068e+12,192.000000,6.510033e+04,1.104500e+05,4.333333,0.003422,3,1,0,0.666667,0.00,12.0,-73,NaN,20.0,-364.333333,NaN,20.0,-364.333333,NaN
1,108129,4830.93,0.0,23688.0,0.0,33052.5,0.0,-1208,-1176.0,9,0.0,25188.615,607500.0,679671.0,24750.0,607500.0,36.0,-164,-134.0,13,0.556485,13039.485,201525.75,257522.250,8250.00,241830.9,15.666667,-713.666667,-639.833333,10.500000,0.185495,78236.910,1209154.5,1545133.5,24750.0,1209154.5,94.0,-4282,-3839.0,63,0.556485,7.075052e+07,6.725932e+10,7.678783e+10,2.041875e+08,7.189037e+10,263.066667,1.330471e+05,1.183198e+05,2.300000,0.103225,6,1,0,1.000000,0.00,36.0,-164,NaN,26.0,-432.666667,NaN,16.4,-614.800000,-713.0
2,122040,6300.00,0.0,0.0,0.0,58545.0,0.0,-2576,-2545.0,10,0.0,15060.735,112500.0,136444.5,9571.5,112500.0,12.0,-301,-271.0,11,0.104691,9623.115,66690.00,72696.375,4785.75,88920.0,8.000000,-1395.750000,-1727.000000,10.750000,0.052345,28869.345,266760.0,290785.5,9571.5,266760.0,24.0,-5583,-5181.0,43,0.104691,2.254097e+07,2.484968e+09,3.239271e+09,4.580681e+07,7.624145e+08,48.000000,1.602844e+06,1.598052e+06,0.250000,0.005480,4,0,0,0.750000,0.25,NaN,-301,NaN,6.0,-1002.333333,-2405.0,8.0,-1395.750000,-2405.0


In [41]:
# Saving
prev_app_new_features.to_csv(os.path.join(PATH_TO_DATA, 'prev_application_agg_features.csv'), index=False)